In [3]:
import os
from pymatgen.io.cif import CifParser
from pymatgen.io.vasp import Poscar

def create_folders_and_poscars(cif_directory):
    """
    为目录中的每个 .cif 文件创建一个文件夹，并生成对应的 POSCAR 文件。

    :param cif_directory: 包含 .cif 文件的目录
    """
    # 遍历目录中的所有文件
    for filename in os.listdir(cif_directory):
        if filename.endswith(".cif"):
            # 分离文件名和扩展名
            basename = os.path.splitext(filename)[0]
            # 创建对应的目录
            new_dir = os.path.join(cif_directory, basename)
            os.makedirs(new_dir, exist_ok=True)
            
            # 读取 .cif 文件并创建结构
            cif_path = os.path.join(cif_directory, filename)
            parser = CifParser(cif_path)
            structure = parser.parse_structures(primitive=True)[0]  # 使用新方法并保留原始行为
            
            # 创建 POSCAR 文件
            poscar = Poscar(structure)
            poscar_path = os.path.join(new_dir, "POSCAR")
            poscar.write_file(poscar_path)
            
            print(f"Created directory {new_dir} and generated POSCAR at {poscar_path}")



# 使用示例
cif_directory = '/home/haoxw/MEOwork/test'
create_folders_and_poscars(cif_directory)


Created directory /home/haoxw/MEOwork/test/Fe3Tc2Ir3Ru8O32_1 and generated POSCAR at /home/haoxw/MEOwork/test/Fe3Tc2Ir3Ru8O32_1/POSCAR


In [4]:
import os
from pymatgen.io.vasp.inputs import Incar, Kpoints, Poscar, Potcar

def generate_vasp_files_for_directory(test_directory):
    for root, dirs, files in os.walk(test_directory):
        for file in files:
            if file == "POSCAR":
                poscar_path = os.path.join(root, file)
                structure = Poscar.from_file(poscar_path).structure

                # 创建自定义的 INCAR
                incar = Incar({
                    "ENCUT": 520,
                    "ISMEAR": -5,  # 使用 Gamma 点方法
                    "SIGMA": 0.05,
                    "IBRION": 2,
                    "ISIF": 3,
                    "NSW": 100
                })

                # 创建 Gamma 点的 KPOINTS
                kpoints = Kpoints.gamma_automatic([3, 3, 3])

                # 创建 POTCAR，假设使用 PBE 赝势
                potcar = Potcar(symbols=[site.species_string for site in structure], functional="PBE")

                # 保存文件
                incar.write_file(os.path.join(root, 'INCAR'))
                kpoints.write_file(os.path.join(root, 'KPOINTS'))
                potcar.write_file(os.path.join(root, 'POTCAR'))

                print(f"Generated VASP files for {poscar_path}")

# 指定 test 目录
test_directory = "/home/haoxw/MEOwork/test"
generate_vasp_files_for_directory(test_directory)


Generated VASP files for /home/haoxw/MEOwork/test/Fe3Tc2Ir3Ru8O32_1/POSCAR


In [5]:
import os
import shutil

def distribute_shell_script(script_path, root_directory):
    """
    将指定的 shell 脚本复制到包含 VASP 输入文件的每个目录中。

    :param script_path: shell 脚本的完整路径
    :param root_directory: 需要遍历的根目录
    """
    # 确保脚本文件存在
    if not os.path.isfile(script_path):
        raise FileNotFoundError(f"The script file '{script_path}' does not exist.")

    # 遍历目录
    for subdir, dirs, files in os.walk(root_directory):
        # 检查是否为 VASP 输入文件目录
        if set(['INCAR', 'KPOINTS', 'POSCAR', 'POTCAR']).issubset(set(files)):
            # 构建目标路径
            target_script_path = os.path.join(subdir, os.path.basename(script_path))
            # 复制文件
            shutil.copy(script_path, target_script_path)
            print(f"Copied '{script_path}' to '{target_script_path}'")

# 使用示例
script_path = '/home/haoxw/MEOwork/vasp-mu01.sh'
root_directory = '/home/haoxw/MEOwork/test'
distribute_shell_script(script_path, root_directory)


Copied '/home/haoxw/MEOwork/vasp-mu01.sh' to '/home/haoxw/MEOwork/test/Fe3Tc2Ir3Ru8O32_1/vasp-mu01.sh'
